<a id="top"></a>
# 03_TRAIN_CONTINUAL — Entrenamiento Continual con **presets**

**Qué hace este notebook:**
- Lanza un **run base** con el método del preset (`configs/presets.yaml`).
- Ejecuta **lotes comparativos** (misma semilla/datos/modelo; cambia solo el método y sus `params`).
- Ofrece un **barrido paramétrico opcional** (grid/variantes).
- **Reevalúa** runs existentes cuando falta/está incompleta `eval_matrix.json`.
- Genera **resúmenes y gráficas** en `outputs/summary/` usando `src/plots.py`.

## ✅ Prerrequisitos
1. Haber generado `data/processed/tasks.json` (y opcional `tasks_balanced.json`) con `01_DATA_QC_PREP` o `01A_PREP_BALANCED`.
2. Si el preset usa **offline** (`use_offline_spikes: true`), haber creado los **H5 v2** con `02_ENCODE_OFFLINE` para el mismo `encoder/T/gain/size/to_gray`.

## ⚠️ Notas
- No mezcles `use_offline_spikes: true` **y** `encode_runtime: true`. El cuaderno aborta si detecta conflicto.
- La carpeta de salida codifica `preset`, `método`, `encoder`, `modelo`, `seed`, etc. (trazabilidad).
- Para gráficas y tablas, se utilizan utilidades centralizadas de `src/plots.py`.

---

<a id="toc"></a>
## 🧭 Índice
1. [Setup del entorno y paths](#sec-01) 
2. [Carga del preset y guardarraíl datos](#sec-02) 
3. [Selección de *tasks* y verificación de datos/H5](#sec-03) 
4. [Factories unificadas: DataLoaders + Modelo + *task_list*](#sec-04)
5. [Ejecución base con el preset](#sec-05)
6. [Comparativa de métodos (lista cerrada)](#sec-06)
7. [Barrido paramétrico (opcional)](#sec-07)
8. [Reevaluación de runs (eval_matrix) — **firma nueva**](#sec-08)
9. [Resumen + gráficas (tablas, leaderboards, plots)](#sec-09)

<a id="sec-01"></a>
## 1) Setup del entorno y paths

**Objetivo:** preparar entorno (ROOT, `sys.path`, device), fijar *hints* de rendimiento e inicializar carpeta de salidas `outputs/`.

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# 1) Setup del entorno y paths
# =============================================================================
import os, sys, torch
from pathlib import Path

# Consejos de estabilidad (WSL/HDF5/CUDA)
os.environ.setdefault("HDF5_USE_FILE_LOCKING", "FALSE")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:64")
os.environ["TRAIN_LOG_ITPS"] = os.environ.get("TRAIN_LOG_ITPS", "1")

try:
    import torch.multiprocessing as mp
    mp.set_sharing_strategy("file_system")
except Exception:
    pass

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

OUT = ROOT / "outputs"
OUT.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("OUT :", OUT)
print("Device:", device)


<a id="sec-02"></a>
## 2) Carga del preset y guardarraíl datos

**Objetivo:** cargar `PRESET` desde `configs/presets.yaml` y derivar:
- Modelo/transform (`img_w/img_h`, `to_gray`).
- Datos/codificación (`encoder`, `T`, `gain`, `seed`).
- Loader (`num_workers`, `prefetch_factor`, `pin_memory`, `persistent_workers`).
- *Augment* y balanceo **online** si procede.
- **Guardarraíl**: aborta si `use_offline_spikes` y `encode_runtime` están ambos a `true`.

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# 2) Carga del preset (configs/presets.yaml)
# =============================================================================
from src.config import load_preset
from src.datasets import ImageTransform, AugmentConfig

PRESET = "fast"  # "fast" para pruebas, "std" estable, "accurate" para resultados
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# ---- Modelo / Transform ----
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"], CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=None,
)

# ---- Datos / Codificación ----
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])
USE_OFFLINE_SPIKES = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE     = bool(CFG["data"].get("encode_runtime", False))

# ---- Loader / Augment / Balanceo online ----
NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)
PREFETCH    = int(CFG["data"].get("prefetch_factor") or 2)
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
    if CFG["data"].get("aug_train") else None

USE_ONLINE_BALANCING = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or CFG.get("prep", {}).get("bins", 50) or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

# Guardarraíl de coherencia
if USE_OFFLINE_SPIKES and RUNTIME_ENCODE:
    raise RuntimeError("Config inválida: use_offline_spikes=True y encode_runtime=True simultáneamente.")

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BALANCING} bins={BAL_BINS} eps={BAL_EPS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")


<a id="sec-03"></a>
## 3) Selección de *tasks* y verificación de datos/H5

**Objetivo:** elegir el fichero de tareas adecuado y comprobar:
- Existencia de `train/val/test.csv` por *run*.
- Si es **offline**, existencia de H5 v2 compatibles con el preset (`encoder/T/gain/size/to_gray`).

Preferencia: `prep.use_balanced_tasks: true` → usa `tasks_balanced.json` si existe; si no, `tasks.json`.

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# 3) Verificación de datos (splits y, si procede, H5)
# =============================================================================
import json
from pathlib import Path as _P

PROC = ROOT / "data" / "processed"
USE_BALANCED = bool(CFG.get("prep", {}).get("use_balanced_tasks", False))
tb_name = (CFG.get("prep", {}).get("tasks_balanced_file_name") or "tasks_balanced.json")
t_name  = (CFG.get("prep", {}).get("tasks_file_name") or "tasks.json")

cand_bal = PROC / tb_name
cand_std = PROC / t_name
TASKS_FILE = cand_bal if (USE_BALANCED and cand_bal.exists()) else cand_std

tasks_json = json.loads(TASKS_FILE.read_text(encoding="utf-8"))
task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

print("Usando tasks:", TASKS_FILE.name)
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# Chequeo coherencia balanced
if USE_BALANCED:
    for t in task_list:
        train_path = _P(tasks_json["splits"][t["name"]]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en modo balanced y encontré '{train_path.name}'."
            )

# Chequeo H5 si offline
if USE_OFFLINE_SPIKES:
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color  = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []
    for t in task_list:
        base = PROC / t["name"]
        for split in ("train", "val", "test"):
            expected = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not expected.exists():
                missing.append(str(expected))
    if missing:
        print("[WARN] Faltan H5 compatibles con el preset. Genera primero con 02_ENCODE_OFFLINE.")
else:
    print("Modo CSV + codificación en runtime (si RUNTIME_ENCODE=True).")

print("OK: verificación de splits/H5.")


<a id="sec-04"></a>
## 4) Factories unificadas: DataLoaders + Modelo + *task_list*

**Objetivo:** construir en **una sola llamada** los componentes coherentes con un `cfg`:
- `build_components_for(cfg, ROOT)` → `tfm, make_loader_fn, make_model_fn`.
- `build_task_list_for(cfg, ROOT)` → `task_list` + `tasks_file`.

Estas *factories* abstraen si trabajas con **H5 offline** o **CSV+runtime**, así como *workers/prefetch/pin/persistent*, *augment*, balanceo online, etc.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# 4) Factories: DataLoaders + Modelo + task_list
# =============================================================================
from src.utils import build_task_list_for, build_components_for

tfm_fac, make_loader_fn, make_model_fn = build_components_for(CFG, ROOT)
task_list_fac, tasks_file_used = build_task_list_for(CFG, ROOT)

print("Tasks file efectivo:", tasks_file_used.name)
print("Factories OK (offline/CSV se resuelve internamente).")


<a id="sec-05"></a>
## 5) Ejecución base con el preset

**Objetivo:** ejecutar **un** experimento con el método del preset (`CFG["continual"]`).

Salida en `outputs/continual_*` con:
- `continual_results.json`, `eval_matrix.(json|csv)`, `forgetting_*.json`,
- `per_task_perf.(json|csv|v2.csv)`,
- `efficiency_summary.json`, `run_row.*`, `method_params.json`…

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# 5) Ejecución base con el preset
# =============================================================================
from src.runner import run_continual

print(
    f"[RUN] preset={PRESET} | method={CFG['continual']['method']} "
    f"| seed={CFG['data']['seed']} | enc={CFG['data']['encoder']} "
    f"| kwargs={CFG['continual'].get('params', {})}"
)
print(f"[MODEL] {MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(
    f"[DATA] T={CFG['data']['T']} gain={CFG['data']['gain']} "
    f"| offline_spikes={CFG['data']['use_offline_spikes']} "
    f"| runtime_encode={CFG['data']['encode_runtime']}"
)
print(
    f"[LOADER] workers={CFG['data']['num_workers']} prefetch={CFG['data']['prefetch_factor']} "
    f"pin={CFG['data']['pin_memory']} persistent={CFG['data']['persistent_workers']} "
    f"| aug={bool(CFG['data']['aug_train'])} | balance_online={CFG['data']['balance_online']}"
)

out_path_base, _ = run_continual(
    task_list=task_list_fac,
    make_loader_fn=make_loader_fn,
    make_model_fn=make_model_fn,
    tfm=tfm_fac,
    cfg=CFG,
    preset_name=PRESET,
    out_root=OUT,
    verbose=True,
)
print("OK:", out_path_base)


<a id="sec-06"></a>
## 6) Comparativa de métodos (lista cerrada)

**Objetivo:** lanzar **varios métodos** cambiando solo `continual.method` y sus `params`, manteniendo fijos datos/modelo/semilla.

- Se clona `CFG` por entrada.
- **Etiqueta** cada run con `cfg_i["naming"]["tag"]` para identificar la variante.
- Ajuste de robustez: para `rehearsal`, se fuerza `persistent_workers=False`.

> La forma de **pasar combinaciones** ha cambiado respecto a versiones antiguas: usa una **lista de dicts `EXPS`** con `method`, `params`, `tag`. El bucle se encarga del resto (no repitas lógica del runner).

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# 6) Comparativa de métodos / variantes
# =============================================================================
from copy import deepcopy
import gc, time
from src.runner import run_continual
from src.utils import build_task_list_for, build_components_for

EXPS = [
    # Baseline
    dict(method="naive", params={}, tag="cmp_naive"),

    # EWC
    dict(method="ewc", params={"lam": 7e8, "fisher_batches": 1000}, tag="cmp_ewc_lam7e8_fb1000"),

    # Rehearsal
    dict(method="rehearsal", params={"buffer_size": 3000, "replay_ratio": 0.10}, tag="cmp_reh_rr10"),
    dict(method="rehearsal", params={"buffer_size": 3000, "replay_ratio": 0.20}, tag="cmp_reh_rr20"),

    # SA-SNN
    dict(method="sa-snn", params={"attach_to":"f6","k":8,"tau":28,"th_min":1.0,"th_max":2.0,"p":2_000_000,
                                  "vt_scale":1.0,"flatten_spatial":False,
                                  "assume_binary_spikes":False,"reset_counters_each_task":False},
         tag="cmp_sa_k8_tau28_p2m"),

    # AS-SNN
    dict(method="as-snn", params={"gamma_ratio":0.25,"lambda_a":1.20,"ema":0.90,
                                  "attach_to":"f6","measure_at":"modules","penalty_mode":"l1",
                                  "do_synaptic_scaling":False},
         tag="cmp_as_soft"),
    dict(method="as-snn", params={"gamma_ratio":0.35,"lambda_a":1.80,"ema":0.95,
                                  "attach_to":"f6","measure_at":"modules","penalty_mode":"l1",
                                  "do_synaptic_scaling":True,"scale_clip":(0.5,2.0),"scale_bias":False},
         tag="cmp_as_scaling"),

    # SCA-SNN
    dict(method="sca-snn", params={"attach_to":"f6","flatten_spatial":False,"num_bins":50,
                                   "anchor_batches":16,"beta":0.60,"bias":0.05,"soft_mask_temp":0.50,
                                   "verbose":False,"log_every":65536},
         tag="cmp_sca_b060_bias005_t050_ab16"),
]

SAFE_DATALOADER_FOR_ALL = False   # Si True, fuerza num_workers=0 para todo
SLEEP_BETWEEN_RUNS_SEC = 1.0

runs_out = []
for idx, exp in enumerate(EXPS, start=1):
    cfg_i = deepcopy(CFG)
    cfg_i["continual"]["method"] = exp["method"]
    cfg_i["continual"]["params"] = exp["params"]
    cfg_i.setdefault("naming", {})
    cfg_i["naming"]["tag"] = exp["tag"]

    # Seguridad dataloader
    if SAFE_DATALOADER_FOR_ALL or ("rehearsal" in exp["method"].lower()):
        cfg_i["data"]["persistent_workers"] = False
    if SAFE_DATALOADER_FOR_ALL:
        cfg_i["data"]["num_workers"] = 0

    # Factories y tasks coherentes con ESTE cfg_i
    tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i, ROOT)
    task_list_i, tasks_file_i = build_task_list_for(cfg_i, ROOT)

    print(
        f"\n=== RUN {idx}/{len(EXPS)} preset={PRESET} | method={exp['method']} "
        f"| enc={cfg_i['data']['encoder']} | tag={exp['tag']} ==="
    )
    try:
        out_dir, _ = run_continual(
            task_list=task_list_i,
            make_loader_fn=make_loader_fn_i,
            make_model_fn=make_model_fn_i,
            tfm=tfm_i,
            cfg=cfg_i,
            preset_name=PRESET,
            out_root=OUT,
            verbose=True,
        )
        runs_out.append(out_dir)
        print("[OK]", out_dir)
    except Exception as e:
        print(f"[ERROR] Falló method={exp['method']} tag={exp['tag']}: {type(e).__name__}: {e}")

    # Limpieza entre runs
    try:
        del tfm_i, make_loader_fn_i, make_model_fn_i, task_list_i
    except Exception:
        pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    time.sleep(SLEEP_BETWEEN_RUNS_SEC)

print("\nHecho:", [str(p) for p in runs_out])


<a id="sec-07"></a>
## 7) Barrido paramétrico (opcional)

**Objetivo:** patrón compacto para probar **variantes de un mismo método** (p.ej., *SCA-SNN* variando `beta` y `bias`).
- Define una **función generadora** de `params` a partir de rejillas.
- Reutiliza el mismo bucle del punto 6 (cambia solo la fuente `EXPS_GRID`).

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# 7) Barrido paramétrico — ejemplo SCA-SNN
# =============================================================================
from itertools import product
from copy import deepcopy
import gc, time
from src.runner import run_continual
from src.utils import build_task_list_for, build_components_for

def sca_params_grid(betas=(0.55, 0.60, 0.65), biases=(0.00, 0.05, 0.10), bins=(50,), anchors=(16,)):
    for beta, bias, nb, ab in product(betas, biases, bins, anchors):
        yield dict(
            method="sca-snn",
            params={"attach_to":"f6","flatten_spatial":False,"num_bins":nb,
                    "anchor_batches":ab,"beta":beta,"bias":bias,"soft_mask_temp":0.50,
                    "verbose":False,"log_every":65536},
            tag=f"grid_sca_b{beta:.2f}_bias{bias:.2f}_bins{nb}_ab{ab}"
        )

# Activa para lanzar el grid:
DO_GRID = False

if DO_GRID:
    EXPS_GRID = list(sca_params_grid())
    runs_out_grid = []
    for idx, exp in enumerate(EXPS_GRID, start=1):
        cfg_i = deepcopy(CFG)
        cfg_i["continual"]["method"] = exp["method"]
        cfg_i["continual"]["params"] = exp["params"]
        cfg_i.setdefault("naming", {})
        cfg_i["naming"]["tag"] = exp["tag"]
        cfg_i["data"]["persistent_workers"] = False  # robustez en grids

        tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i, ROOT)
        task_list_i, tasks_file_i = build_task_list_for(cfg_i, ROOT)

        print(f"\n=== GRID {idx}/{len(EXPS_GRID)} {exp['tag']} ===")
        try:
            out_dir, _ = run_continual(
                task_list=task_list_i,
                make_loader_fn=make_loader_fn_i,
                make_model_fn=make_model_fn_i,
                tfm=tfm_i,
                cfg=cfg_i,
                preset_name=PRESET,
                out_root=OUT,
                verbose=True,
            )
            runs_out_grid.append(out_dir)
            print("[OK]", out_dir)
        except Exception as e:
            print(f"[ERROR] {exp['tag']}: {type(e).__name__}: {e}")

        try:
            del tfm_i, make_loader_fn_i, make_model_fn_i, task_list_i
        except Exception:
            pass
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        time.sleep(0.5)

    print("\nGrid hecho:", [str(p) for p in runs_out_grid])
else:
    print("Grid desactivado (DO_GRID=False).")


<a id="sec-08"></a>
## 8) Reevaluación de runs (eval_matrix) — **firma nueva**

**Objetivo:** reconstruir `eval_matrix.json`/`forgetting.*` cuando:
- Falta el fichero, o
- Tiene forma vacía/incompleta (p. ej., columnas finales `NaN`).

> **Importante:** `reevaluate_only(...)` ahora **requiere** los *factories* y `task_list` además de `out_dir`, pues necesita reconstruir los *loaders* coherentes con el `cfg`. Este cuaderno ya prepara todo correctamente.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# 8) Reevaluación de runs (corrige firma y detección)
# =============================================================================
from pathlib import Path
import json, numpy as np
from src.runner import reevaluate_only

def _needs_reeval(run_dir: Path) -> bool:
    jf = run_dir / "eval_matrix.json"
    if not jf.exists():
        return True
    try:
        j = json.loads(jf.read_text(encoding="utf-8"))
        tasks = j.get("tasks") or []
        M = j.get("mae_matrix") or []
        A = np.array(M, dtype=float)
        if A.ndim != 2 or A.shape[0] != len(tasks) or A.shape[1] == 0:
            return True
        last_col = A[:, -1]
        return not np.isfinite(last_col).any()
    except Exception:
        return True

# Factories y tasks de referencia (usamos el preset actual)
tfm_r, make_loader_fn_r, make_model_fn_r = build_components_for(CFG, ROOT)
task_list_r, _ = build_task_list_for(CFG, ROOT)

targets = []
for p in sorted(OUT.glob("continual_*")):
    if p.is_dir() and _needs_reeval(p):
        targets.append(p)

print(f"[INFO] Runs a reevaluar: {len(targets)}")
for rd in targets:
    print(" -", rd.name)
    reevaluate_only(
        out_dir=rd,
        task_list=task_list_r,
        make_loader_fn=make_loader_fn_r,
        make_model_fn=make_model_fn_r,
        tfm=tfm_r,
        cfg=CFG,
        preset_name=PRESET,
        verbose=True,
    )
print("[OK] Reevaluación terminada.")


<a id="sec-09"></a>
## 9) Resumen + gráficas (tablas, leaderboards, plots)

**Objetivo:** centralizar reporting usando `src/plots.py`:
- Tabla consolidada `results_table.csv` a partir de `outputs/`.
- *Leaderboards* y agregados por método: `export_leaderboards(...)`.
- Gráficas globales: `plot_across_runs(...)`.
- Historias por run/tarea: `plot_loss_curves_all_runs(...)` (opcional).
- Heatmap `eval_matrix` y reparto de **emisiones por tarea** (si hay telemetría).

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# 9) Resumen y gráficas
# =============================================================================
from pathlib import Path
import pandas as pd
from src.results_io import build_results_table
from src.plots import (
    export_leaderboards, plot_across_runs, plot_loss_curves_all_runs,
    plot_mae_curves_for_run, plot_eval_matrix_heatmap, plot_energy_by_task
)

summary_dir = OUT / "summary"
summary_dir.mkdir(parents=True, exist_ok=True)

# 9.1 Tabla consolidada (desde outputs/*)
df = build_results_table(OUT)
display(df.head(10))
tbl_path = summary_dir / "results_table.csv"
df.to_csv(tbl_path, index=False)
print("[OK] Tabla consolidada:", tbl_path)

# 9.2 Leaderboards y agregados (preset actual por defecto)
ld_paths = export_leaderboards(df, summary_dir / "leaderboards", preset=PRESET, topN=6)
print("[OK] Leaderboards/agregados:", ld_paths)

# 9.3 Gráficas globales (MAE final por tarea, olvido, emisiones, trade-off)
plots_dir = plot_across_runs(df, summary_dir / "plots_global")
print("[OK] Gráficas globales:", plots_dir)

# 9.4 Curvas de validación por run/tarea (opcional)
DO_CURVES_ALL = False
if DO_CURVES_ALL:
    curves_dir = plot_loss_curves_all_runs(OUT, summary_dir, smooth_window=3)
    print("[OK] Curvas por run:", curves_dir)

# 9.5 Para el último run por fecha: heatmap eval_matrix + energía por tarea (si hay)
runs_sorted = sorted(OUT.glob("continual_*"), key=lambda p: p.stat().st_mtime, reverse=True)
if runs_sorted:
    last_run = runs_sorted[0]
    print("Último run:", last_run.name)
    try:
        plot_eval_matrix_heatmap(last_run, summary_dir / "by_run")
        plot_energy_by_task(last_run, summary_dir / "by_run")
        plot_mae_curves_for_run(last_run, summary_dir / "by_run", smooth_window=3)
        print("[OK] Plots por run:", summary_dir / "by_run" / last_run.name)
    except Exception as e:
        print("[WARN] No se pudieron generar algunos plots por run:", e)
else:
    print("[INFO] No hay carpetas 'continual_*' en outputs/ todavía.")
